In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from scipy.stats import chi2_contingency
import matplotlib.colors as colors

from sklearn.model_selection import train_test_split  
from sklearn.linear_model import LogisticRegression  
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_score, cross_validate

from sklearn.metrics import make_scorer, f1_score, roc_auc_score, precision_score, recall_score, silhouette_score
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from scipy.stats import uniform, loguniform, randint

from imblearn.over_sampling import SMOTE

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.cluster import KMeans


pd.set_option("display.max_columns", None)

In [ ]:
# Ler do dataframe original
df = pd.read_csv("../data/dados.csv", encoding="ISO-8859-1", sep=";", low_memory=False)
print(f'Linhas, Colunas: {df.shape} \n')
print(f'Informações: \n')
print(f'{df.info()}\n')


In [ ]:
#Overview do dataset
df.info() 

In [ ]:
#Carrega o dataset de taxas de rendimento
df_tx = pd.read_excel("../data/tx_rend.xlsx", skiprows = 8, nrows = 129307)
print(f'Linhas, Colunas: {df_tx.shape} \n')
print(f'Informações: \n')
print(f'{df_tx.info()}\n')


In [ ]:
#Dados os datatypes criados a partir dos dois datasets:
df.info()
df_tx.info()

In [ ]:
# Criar a lista de entidade para cada uma das bases
lista_co_entidades_microdados = pd.Series(df['CO_ENTIDADE'].to_list())
lista_co_entidades_tx_rend = pd.Series(df_tx['CO_ENTIDADE'].to_list())
print(f'O dataset de microdados possui {len(lista_co_entidades_microdados)} códigos de entidade, enquanto o dataset de taxa de rendimentos possui {len(lista_co_entidades_tx_rend)} códigos.\n')

#Obtendo os códigos presentes em um dataset mas ausente em outro
codigos_diferentes = [cod for cod in lista_co_entidades_microdados.to_list() if cod not in lista_co_entidades_tx_rend.to_list()]


In [ ]:
#Total de códigos diferentes
len(codigos_diferentes)

In [ ]:
#Criando um novo dataframe filtrando os microdados apenas para aqueles que não possuem CO_ENTIDADE que não 
#tem correspondente na base de taxa de rendimento
df_microdados = df.query('CO_ENTIDADE not in @codigos_diferentes').reset_index(drop=True)
df_microdados.info()

In [ ]:
#Seleciona as colunas relevantes a partir de seus índices
indices_col_tx = [5,45,57]
df_tx_filtrado = df_tx.iloc[:,indices_col_tx]

In [ ]:
#Renomeia as colunas do Dataframe alvo para ficarem com valores mais claros
df_tx_filtrado = df_tx_filtrado.rename(columns={"3_CAT_FUN" : "ABAND_FUND", "3_CAT_MED" : "ABAND_MED"})
df_tx_filtrado

In [ ]:
#Filtra as colunas selecionadas apenas do dataset de microdados:
indices_col_microdados = ['CO_ENTIDADE', 'IN_AGUA_POTAVEL', 'IN_ENERGIA_INEXISTENTE', 'IN_ESGOTO_INEXISTENTE', 'IN_BANHEIRO', 'IN_BIBLIOTECA_SALA_LEITURA', 'IN_COZINHA', 'IN_LABORATORIO_CIENCIAS', 'IN_LABORATORIO_INFORMATICA', 'IN_PARQUE_INFANTIL', 'IN_QUADRA_ESPORTES',
                         'IN_REFEITORIO', 'IN_LABORATORIO_EDUC_PROF', 'IN_SALA_MULTIUSO', 'IN_SALA_REPOUSO_ALUNO', 'IN_ACESSIBILIDADE_INEXISTENTE', 'QT_SALAS_UTILIZADAS', 'QT_SALAS_UTILIZA_CLIMATIZADAS', 'IN_DESKTOP_ALUNO', 'IN_COMP_PORTATIL_ALUNO', 'IN_TABLET_ALUNO', 'IN_INTERNET', 'IN_INTERNET_APRENDIZAGEM',
                         'IN_INTERNET_COMUNIDADE', 'IN_PROF_BIBLIOTECARIO', 'IN_PROF_SAUDE', 'IN_ALIMENTACAO', 
                          'IN_ESPACO_ATIVIDADE', 'IN_FUND', 'IN_MED']
df_microdados_filtrado = df_microdados.loc[:,indices_col_microdados]
df_microdados_filtrado

In [ ]:
# Salva os dataframes em arquivos, para um checkpoint
df_tx_filtrado.to_pickle("../data/tx_aband.pk1")
df_microdados_filtrado.to_feather("../data/microdados.feather")
df_tx_filtrado.info()
df_microdados_filtrado.info()

In [ ]:
#Carrega do backup
df_tx_use = pd.read_pickle("../data/tx_aband.pk1")
df_mc_use = pd.read_feather("../data/microdados.feather")

df_microdados = df_mc_use.copy()
df_tx = df_tx_use.copy()
df_tx.info()
df_microdados.info()



In [ ]:

# Seleção das colunas binárias
columns_dt = ['IN_AGUA_POTAVEL', 'IN_ENERGIA_INEXISTENTE', 'IN_BANHEIRO', 'IN_ACESSIBILIDADE_INEXISTENTE']

# Configurar o estilo do seaborn para gráficos mais bonitos
sns.set_theme(style="whitegrid")

# Função para mapear 0 e 1 para 'negativo' e 'positivo'
def map_labels(value):
    return 'negativo' if value == 0 else 'positivo'

# Preparar os dados para o gráfico agrupado
df_melted = pd.DataFrame()

for col in columns_dt:
    temp_df = df_microdados[[col]].copy()
    temp_df['Feature'] = col
    temp_df['Value'] = temp_df[col].map(map_labels)
    df_melted = pd.concat([df_melted, temp_df], axis=0)

# Definindo a paleta de cores personalizada
palette = {'negativo': 'red', 'positivo': 'green'}

# Plotar o gráfico de barras agrupado
plt.figure(figsize=(10, 6))

ax = sns.countplot(data=df_melted, x='Feature', hue='Value', palette=palette, )

# Adicionar rótulos de contagem em cima das barras
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='baseline', fontsize=12, color='black', xytext=(0, 5),
                textcoords='offset points')

plt.title('Contagem de valores positivos e negativos para cada característica')
plt.xlabel('Característica')
plt.ylabel('Contagem')
plt.legend(title='Valor')
plt.tight_layout()
plt.show()


In [ ]:

# Supondo que df_microdados seja o seu DataFrame original
# Seleção das colunas binárias
columns_dt = ['IN_LABORATORIO_INFORMATICA', 'IN_DESKTOP_ALUNO',
       'IN_COMP_PORTATIL_ALUNO', 'IN_TABLET_ALUNO', 'IN_INTERNET',
       'IN_INTERNET_APRENDIZAGEM', 'IN_INTERNET_COMUNIDADE',
       ]

# Configurar o estilo do seaborn para gráficos mais bonitos
sns.set_theme(style="whitegrid")

# Função para mapear 0 e 1 para 'negativo' e 'positivo'
def map_labels(value):
    return 'negativo' if value == 0 else 'positivo'

# Preparar os dados para o gráfico agrupado
df_melted = pd.DataFrame()

for col in columns_dt:
    temp_df = df_microdados[[col]].copy()
    temp_df['Feature'] = col
    temp_df['Value'] = temp_df[col].map(map_labels)
    df_melted = pd.concat([df_melted, temp_df], axis=0)

# Definindo a paleta de cores personalizada
palette = {'negativo': 'red', 'positivo': 'green'}

# Plotar o gráfico de barras agrupado
plt.figure(figsize=(18, 8))

ax = sns.countplot(data=df_melted, x='Feature', hue='Value', palette=palette, )

# Adicionar rótulos de contagem em cima das barras
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='baseline', fontsize=12, color='black', xytext=(0, 5),
                textcoords='offset points')

plt.title('Contagem de valores positivos e negativos para cada característica')
plt.xlabel('Característica')
plt.ylabel('Contagem')
plt.legend(title='Valor')
plt.tight_layout()
plt.show()


In [ ]:

# Supondo que df_microdados seja o seu DataFrame original
# Seleção das colunas binárias
columns_dt = ['IN_BIBLIOTECA_SALA_LEITURA', 'IN_PROF_BIBLIOTECARIO']

# Configurar o estilo do seaborn para gráficos mais bonitos
sns.set_theme(style="whitegrid")

# Função para mapear 0 e 1 para 'negativo' e 'positivo'
def map_labels(value):
    return 'negativo' if value == 0 else 'positivo'

# Preparar os dados para o gráfico agrupado
df_melted = pd.DataFrame()

for col in columns_dt:
    temp_df = df_microdados[[col]].copy()
    temp_df['Feature'] = col
    temp_df['Value'] = temp_df[col].map(map_labels)
    df_melted = pd.concat([df_melted, temp_df], axis=0)

# Definindo a paleta de cores personalizada
palette = {'negativo': 'red', 'positivo': 'green'}

# Plotar o gráfico de barras agrupado
plt.figure(figsize=(10, 6))

ax = sns.countplot(data=df_melted, x='Feature', hue='Value', palette=palette, )

# Adicionar rótulos de contagem em cima das barras
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='baseline', fontsize=12, color='black', xytext=(0, 5),
                textcoords='offset points')

plt.title('Contagem de valores positivos e negativos para cada característica')
plt.xlabel('Característica')
plt.ylabel('Contagem')
plt.legend(title='Valor')
plt.tight_layout()
plt.show()


In [ ]:
#Produz um dataframe unindo a partir da coluna de CO_ENTIDADE
df_microdados = pd.merge(df_microdados, df_tx, on="CO_ENTIDADE")
df_microdados.info()

In [ ]:
#Dividir as bases que tem fund e as que não tem IN_FUND e IN_MED
df_microdados_fund = df_microdados.query('IN_FUND == 1').reset_index(drop=True)
df_microdados_med = df_microdados.query('IN_MED == 1').reset_index(drop=True)

#Remover as colunas de indicador já que a divisão já foi feita
df_microdados_fund = df_microdados_fund.drop(['IN_FUND', 'IN_MED', 'ABAND_MED' ], axis =1)
df_microdados_med = df_microdados_med.drop(['IN_FUND', 'IN_MED', 'ABAND_FUND' ], axis =1)

df_microdados_fund['ABAND_FUND'] = df_microdados_fund['ABAND_FUND'].replace('--', 0)
df_microdados_med['ABAND_MED'] = df_microdados_med['ABAND_MED'].replace('--', 0)
df_microdados_fund['ABAND_FUND'] = df_microdados_fund['ABAND_FUND'].astype('float')
df_microdados_med['ABAND_MED'] = df_microdados_med['ABAND_MED'].astype('float')

In [ ]:
df_microdados_fund.head()

In [ ]:
df_microdados_med.info(max_cols =1)

In [ ]:
#Estatísticas básicas das variáveis instituições fundamental
df_microdados_fund.describe(include = 'all')

In [ ]:
#Estatísticas básicas das variáveis instituições med
df_microdados_med.describe(include = 'all')

In [ ]:
#Analise descritiva para a variável alvo fundamental
df_microdados_fund['ABAND_FUND'].describe()

In [ ]:
#Analise descritiva para a variável alvo médio
df_microdados_med['ABAND_MED'].describe()

In [ ]:
sns.set_theme()
plt.figure(figsize=(24,12))
plt.subplot(2,1,1)
sns.boxplot(data=df_microdados_fund, x='ABAND_FUND', color='skyblue')
plt.title("Taxa de abandono fundamental")
plt.xticks(range(0,101,5))

plt.subplot(2,1,2)
sns.boxplot(data=df_microdados_med, x='ABAND_MED', color='skyblue')
plt.title("Taxa de abandono médio")
plt.xticks(range(0,101,5))

plt.show()


In [ ]:
# Lista para armazenar as colunas relevantes
relevant_columns_fund = []
irelevant_columns_fund = []
#Itera sobre as colunas do dataframe para criar uma tabela de contigência entre cada coluna e a variável alvo
for column in df_microdados_fund.columns:
    contigency_table = pd.crosstab(df_microdados_fund[column],df_microdados_fund['ABAND_FUND'])
    _, p, _, _ = chi2_contingency(contigency_table)
    
    if p < 0.05:
        relevant_columns_fund.append(column)
    else:
        irelevant_columns_fund.append(column)
    
print(relevant_columns_fund)
print(irelevant_columns_fund)
    

In [ ]:
# Lista para armazenar as colunas relevantes
relevant_columns_med = []
#Itera sobre as colunas do dataframe para criar uma tabela de contigência entre cada coluna e a variável alvo
for column in df_microdados_med.columns:
    contigency_table = pd.crosstab(df_microdados_med[column],df_microdados_med['ABAND_MED'])
    _, p, _, _ = chi2_contingency(contigency_table)
    
    if p < 0.05:
        relevant_columns_med.append(column)
    
print(relevant_columns_med)

In [ ]:
common_coluns = [i for i in relevant_columns_fund if i in relevant_columns_med]

print (f'Colunas em comum: {common_coluns}')

In [ ]:
#Reorganizando os DFs com apenas as colunas relevantes
relevant_columns_fund = ['CO_ENTIDADE'] + common_coluns + ['ABAND_FUND']
relevant_columns_med = ['CO_ENTIDADE'] + common_coluns + ['ABAND_MED']
print(f'Relevant coluns for fund: {relevant_columns_fund} \n Relevant coluns for med: {relevant_columns_med}')
#df_microdados_fund = df_microdados_fund[relevant_columns_fund]
#df_microdados_med = df_microdados_med[relevant_columns_med]

len(df_microdados_med.columns)
#len(df_microdados_fund.columns)

In [ ]:
df_microdados_fund = df_microdados_fund[relevant_columns_fund]
df_microdados_med = df_microdados_med[relevant_columns_med]

In [ ]:
#Visualizando os valores únicos
unique_target_fund = sorted(df_microdados_fund['ABAND_FUND'].unique())
unique_target_med = sorted(df_microdados_med['ABAND_MED'].unique())

print(f'Distinct fund: {unique_target_fund} \n Distinct med: {unique_target_med}')

In [ ]:
#Encontrando os percentis com valores significativos
final = {}
for i in np.arange(0,1,0.01) :
    
    if df_microdados_fund['ABAND_FUND'].quantile(i) > 0:
        final[i] = df_microdados_fund['ABAND_FUND'].quantile(i)
print(final)

print(len(final.keys()))



In [ ]:
#Encontrando os percentis com valores significativos
final = {}
for i in np.arange(0,1,0.01) :
    
    if df_microdados_med['ABAND_MED'].quantile(i) > 0:
        final[i] = df_microdados_med['ABAND_MED'].quantile(i)
print(final)
print(len(final.keys()))




In [ ]:
#Definição dos limites conforme encontrado na análise
limites_bins_fund = [-0.1, 0.1, 0.7, 1.8, 4.1, 100]
limites_bins_med = [-0.1, 0.2, 2.1, 5.4, 10.6, 100]

In [ ]:
#Construção dos gráficos para visualizar a distribuição dentro dos dataframes
ordinal_classes = ['Muito Baixo', 'Baixo', 'Médio', 'Alto', 'Muito Alto']

def gerar_grafico_aband_fund():
    
    categorias = pd.cut(df_microdados_fund["ABAND_FUND"], limites_bins_fund, labels=ordinal_classes)
    cores = ['lightskyblue', 'lightgreen', 'gold','lightcoral', 'darkred']
    cmap = colors.LinearSegmentedColormap.from_list("", cores)

    df_microdados_fund_agrupado = df_microdados_fund.groupby(categorias).size().reset_index().rename(columns={0: 'CONTAGEM'})
    sns.barplot(x='ABAND_FUND', y='CONTAGEM', data=df_microdados_fund_agrupado, hue='ABAND_FUND', palette=cores)

    plt.xlabel('Categoria')
    plt.ylabel('Número de Entradas')
    plt.title('Distribuição por Categoria de ABAND_FUND')

def gerar_grafico_aband_med():
    
    categorias = pd.cut(df_microdados_med["ABAND_MED"], limites_bins_med, labels=ordinal_classes)
    cores = ['lightskyblue', 'lightgreen', 'gold','lightcoral', 'darkred']
    cmap = colors.LinearSegmentedColormap.from_list("", cores)

    df_microdados_fund_agrupado = df_microdados_fund.groupby(categorias).size().reset_index().rename(columns={0: 'CONTAGEM'})
    sns.barplot(x='ABAND_MED', y='CONTAGEM', data=df_microdados_fund_agrupado, hue='ABAND_MED', palette=cores)

    plt.xlabel('Categoria')
    plt.ylabel('Número de Entradas')
    plt.title('Distribuição por Categoria de ABAND_MED')


# Gerar os gráficos em figuras separadas
plt.figure(figsize=(20, 6))  # Definir tamanho da figura (opcional)

# Gerar e posicionar o primeiro gráfico
subplot1 = plt.subplot(1, 2, 1)  # Criar subplot na posição 1, 2 (linha 1, coluna 1)
gerar_grafico_aband_fund()


# Gerar e posicionar o segundo gráfico
subplot2 = plt.subplot(1, 2, 2)  # Criar subplot na posição 1, 2 (linha 1, coluna 2)
gerar_grafico_aband_med()

plt.tight_layout()

# Apresentar os gráficos na tela
plt.show()

In [ ]:
df_microdados_fund['CLASSE_ABAND'] = pd.cut(df_microdados_fund['ABAND_FUND'], bins=limites_bins_fund, labels=ordinal_classes)
df_microdados_med['CLASSE_ABAND'] = pd.cut(df_microdados_med['ABAND_MED'], bins=limites_bins_med, labels=ordinal_classes)

# Regressão Logística


In [ ]:

X = df_microdados_fund.drop(columns=['CO_ENTIDADE', 'ABAND_FUND', 'CLASSE_ABAND'])
y = df_microdados_fund['CLASSE_ABAND']

# Cria um objeto SMOTE
smote = SMOTE()

# Gera amostras sintéticas para a classe minoritária
X_resampled, y_resampled = smote.fit_resample(X, y)

# Encode labels
label_encoder = LabelEncoder()
y_resampled = pd.Series(label_encoder.fit_transform(y_resampled))

# Parâmetros para o RandomizedSearch
param_distributions = {
    'logistic__C': loguniform(1e-4, 1e4),
    'logistic__penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'logistic__solver': ['saga'],  # saga supports l1, l2, and elasticnet
    'logistic__l1_ratio': uniform(0, 1)  # Only used if penalty is elasticnet
}

# Criação do pipeline de pré-processamento e modelo
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logistic', LogisticRegression(multi_class="multinomial", max_iter=10000, class_weight='balanced'))
])

print(f'Using pipeline: {pipeline}')
# RandomizedSearchCV para LogisticRegression com pipeline
random_search = RandomizedSearchCV(pipeline, param_distributions, n_iter=50, cv=5, scoring='f1_weighted')
random_search.fit(X_resampled, y_resampled)

best_model = random_search.best_estimator_

print(f'Melhor modelo encontrado: {best_model}')
# Avaliação adicional usando cross_val_score com mais dobras (e.g., 10 dobras)
scoring = {
    'accuracy': 'accuracy',
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score, average='weighted'),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
    'roc_auc_ovr': make_scorer(roc_auc_score, multi_class='ovr', average='weighted', needs_proba=True)
}

# Stratified K-Fold Cross Validation
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Usar cross_val_score para calcular as métricas de desempenho
scores = cross_val_score(best_model, X_resampled, y_resampled, cv=skfold, scoring='f1_weighted')

# Cálculo das métricas
mean_f1 = np.mean(scores)
print("Média de F1-score:", mean_f1)

# Avaliação de outras métricas
results = cross_validate(best_model, X_resampled, y_resampled, cv=skfold, scoring=scoring)

mean_accuracy = np.mean(results['test_accuracy'])
mean_precision = np.mean(results['test_precision_weighted'])
mean_recall = np.mean(results['test_recall_weighted'])
mean_auc = np.mean(results['test_roc_auc_ovr'])

print("Média de Accuracy:", mean_accuracy)
print("Média de Precision:", mean_precision)
print("Média de Recall:", mean_recall)
print("Média de AUC-ROC:", mean_auc)




In [ ]:

X = df_microdados_fund.drop(columns=['CO_ENTIDADE', 'ABAND_FUND', 'CLASSE_ABAND'])
y = df_microdados_fund['CLASSE_ABAND']

# Cria um objeto SMOTE
smote = SMOTE()

# Gera amostras sintéticas para a classe minoritária
X_resampled, y_resampled = smote.fit_resample(X, y)

# Encode labels
label_encoder = LabelEncoder()
y_resampled = pd.Series(label_encoder.fit_transform(y_resampled))

# Parâmetros para o RandomizedSearch
param_distributions = {
    'C': np.logspace(-4, 4, 1000),  # Variação maior para C
    'penalty': ['elasticnet'],  # Apenas 'elasticnet' é usado
    'solver': ['saga'],  # Apenas 'saga' é usado
    'l1_ratio': np.linspace(0, 1, 1000)  # Variação maior para l1_ratio
}

# RandomizedSearchCV para LogisticRegression
random_search = RandomizedSearchCV(LogisticRegression(multi_class="multinomial", max_iter=100000, class_weight='balanced'), param_distributions, n_iter=50, cv=6, scoring='f1_weighted', random_state=42)
random_search.fit(X_resampled, y_resampled)

best_model = random_search.best_estimator_

# Avaliação adicional usando cross_validate com mais dobras (e.g., 10 dobras)
scoring = {
    'accuracy': 'accuracy',
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score, average='weighted'),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
    'roc_auc_ovr': make_scorer(roc_auc_score, multi_class='ovr', average='weighted', needs_proba=True)
}

# Stratified K-Fold Cross Validation
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Usar cross_validate para calcular as métricas de desempenho
results = cross_validate(best_model, X_resampled, y_resampled, cv=skfold, scoring=scoring)

# Cálculo das médias das métricas
mean_accuracy = np.mean(results['test_accuracy'])
mean_precision = np.mean(results['test_precision_weighted'])
mean_recall = np.mean(results['test_recall_weighted'])
mean_f1 = np.mean(results['test_f1_weighted'])
mean_auc = np.mean(results['test_roc_auc_ovr'])

print("Média de Accuracy:", mean_accuracy)
print("Média de Precision:", mean_precision)
print("Média de Recall:", mean_recall)
print("Média de F1-score:", mean_f1)
print("Média de AUC-ROC:", mean_auc)

print(best_model)


In [ ]:

X = df_microdados_med.drop(columns=['CO_ENTIDADE', 'ABAND_MED', 'CLASSE_ABAND'])
y = df_microdados_med['CLASSE_ABAND']

# Cria um objeto SMOTE
smote = SMOTE()

# Gera amostras sintéticas para a classe minoritária
X_resampled, y_resampled = smote.fit_resample(X, y)

# Encode labels
label_encoder = LabelEncoder()
y_resampled = pd.Series(label_encoder.fit_transform(y_resampled))

# Parâmetros para o RandomizedSearch
param_distributions = {
    'C': np.logspace(-4, 4, 1000),  # Variação maior para C
    'penalty': ['elasticnet'],  # Apenas 'elasticnet' é usado
    'solver': ['saga'],  # Apenas 'saga' é usado
    'l1_ratio': np.linspace(0, 1, 1000)  # Variação maior para l1_ratio
}

# RandomizedSearchCV para LogisticRegression
random_search = RandomizedSearchCV(LogisticRegression(multi_class="multinomial", max_iter=100000, class_weight='balanced'), param_distributions, n_iter=50, cv=6, scoring='f1_weighted', random_state=13)
random_search.fit(X_resampled, y_resampled)

best_model = random_search.best_estimator_

# Avaliação adicional usando cross_validate com mais dobras (e.g., 10 dobras)
scoring = {
    'accuracy': 'accuracy',
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score, average='weighted'),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
    'roc_auc_ovr': make_scorer(roc_auc_score, multi_class='ovr', average='weighted', needs_proba=True)
}

# Stratified K-Fold Cross Validation
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=13)

# Usar cross_validate para calcular as métricas de desempenho
results = cross_validate(best_model, X_resampled, y_resampled, cv=skfold, scoring=scoring)

# Cálculo das médias das métricas
mean_accuracy = np.mean(results['test_accuracy'])
mean_precision = np.mean(results['test_precision_weighted'])
mean_recall = np.mean(results['test_recall_weighted'])
mean_f1 = np.mean(results['test_f1_weighted'])
mean_auc = np.mean(results['test_roc_auc_ovr'])

print("Média de Accuracy:", mean_accuracy)
print("Média de Precision:", mean_precision)
print("Média de Recall:", mean_recall)
print("Média de F1-score:", mean_f1)
print("Média de AUC-ROC:", mean_auc)

print(best_model)


# Árvore de Decisão

In [ ]:

X_dt = df_microdados_fund.drop(columns=['CO_ENTIDADE', 'ABAND_FUND', 'CLASSE_ABAND'])
y_dt = df_microdados_fund['CLASSE_ABAND']

# Cria um objeto SMOTE
smote_dt = SMOTE()

# Gera amostras sintéticas para a classe minoritária
X_resampled_dt, y_resampled_dt = smote_dt.fit_resample(X_dt, y_dt)

# Encode labels
label_encoder_dt = LabelEncoder()
y_resampled_dt = pd.Series(label_encoder_dt.fit_transform(y_resampled_dt))

# Parâmetros para o RandomizedSearch
param_distributions_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(5, 100),  # Ampliando o intervalo para max_depth
    'min_samples_split': randint(2, 100),  # Ampliando o intervalo para min_samples_split
    'min_samples_leaf': randint(1, 50),  # Mantendo o intervalo para min_samples_leaf
    'max_features': ['auto', 'sqrt', 'log2', None],
    'class_weight': ['balanced', None],
    'min_impurity_decrease': [0.0, 0.1, 0.2, 0.3],  # Mantendo as opções para min_impurity_decrease
    'splitter': ['best', 'random'],  # Mantendo as opções para splitter
}

# RandomizedSearchCV para DecisionTreeClassifier
random_search_dt = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions_dt, n_iter=100, cv=5, scoring='f1_weighted', random_state=13)
random_search_dt.fit(X_resampled_dt, y_resampled_dt)

best_model_dt = random_search_dt.best_estimator_

# Avaliação adicional usando cross_validate com mais dobras (e.g., 10 dobras)
scoring_dt = {
    'accuracy': 'accuracy',
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score, average='weighted'),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
    'roc_auc_ovr': make_scorer(roc_auc_score, multi_class='ovr', average='weighted', needs_proba=True)
}

# Stratified K-Fold Cross Validation
skfold_dt = StratifiedKFold(n_splits=10, shuffle=True, random_state=13)

# Usar cross_validate para calcular as métricas de desempenho
results_dt = cross_validate(best_model_dt, X_resampled_dt, y_resampled_dt, cv=skfold_dt, scoring=scoring_dt)

# Cálculo das médias das métricas
mean_accuracy_dt = np.mean(results_dt['test_accuracy'])
mean_precision_dt = np.mean(results_dt['test_precision_weighted'])
mean_recall_dt = np.mean(results_dt['test_recall_weighted'])
mean_f1_dt = np.mean(results_dt['test_f1_weighted'])
mean_auc_dt = np.mean(results_dt['test_roc_auc_ovr'])

print("Média de Accuracy:", mean_accuracy_dt)
print("Média de Precision:", mean_precision_dt)
print("Média de Recall:", mean_recall_dt)
print("Média de F1-score:", mean_f1_dt)
print("Média de AUC-ROC:", mean_auc_dt)

print(best_model_dt)


In [ ]:

X_dt = df_microdados_med.drop(columns=['CO_ENTIDADE', 'ABAND_MED', 'CLASSE_ABAND'])
y_dt = df_microdados_med['CLASSE_ABAND']

# Cria um objeto SMOTE
smote_dt = SMOTE()

# Gera amostras sintéticas para a classe minoritária
X_resampled_dt, y_resampled_dt = smote_dt.fit_resample(X_dt, y_dt)

# Encode labels
label_encoder_dt = LabelEncoder()
y_resampled_dt = pd.Series(label_encoder_dt.fit_transform(y_resampled_dt))

# Parâmetros para o RandomizedSearch
param_distributions_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(5, 100),  # Ampliando o intervalo para max_depth
    'min_samples_split': randint(2, 100),  # Ampliando o intervalo para min_samples_split
    'min_samples_leaf': randint(1, 50),  # Mantendo o intervalo para min_samples_leaf
    'max_features': ['auto', 'sqrt', 'log2', None],
    'class_weight': ['balanced', None],
    'min_impurity_decrease': [0.0, 0.1, 0.2, 0.3],  # Mantendo as opções para min_impurity_decrease
    'splitter': ['best', 'random'],  # Mantendo as opções para splitter
}

# RandomizedSearchCV para DecisionTreeClassifier
random_search_dt = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions_dt, n_iter=100, cv=5, scoring='f1_weighted', random_state=13)
random_search_dt.fit(X_resampled_dt, y_resampled_dt)

best_model_dt = random_search_dt.best_estimator_

# Avaliação adicional usando cross_validate com mais dobras (e.g., 10 dobras)
scoring_dt = {
    'accuracy': 'accuracy',
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score, average='weighted'),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
    'roc_auc_ovr': make_scorer(roc_auc_score, multi_class='ovr', average='weighted', needs_proba=True)
}

# Stratified K-Fold Cross Validation
skfold_dt = StratifiedKFold(n_splits=10, shuffle=True, random_state=13)

# Usar cross_validate para calcular as métricas de desempenho
results_dt = cross_validate(best_model_dt, X_resampled_dt, y_resampled_dt, cv=skfold_dt, scoring=scoring_dt)

# Cálculo das médias das métricas
mean_accuracy_dt = np.mean(results_dt['test_accuracy'])
mean_precision_dt = np.mean(results_dt['test_precision_weighted'])
mean_recall_dt = np.mean(results_dt['test_recall_weighted'])
mean_f1_dt = np.mean(results_dt['test_f1_weighted'])
mean_auc_dt = np.mean(results_dt['test_roc_auc_ovr'])

print("Média de Accuracy:", mean_accuracy_dt)
print("Média de Precision:", mean_precision_dt)
print("Média de Recall:", mean_recall_dt)
print("Média de F1-score:", mean_f1_dt)
print("Média de AUC-ROC:", mean_auc_dt)

print(best_model_dt)


In [ ]:
best_model_dt.get_params()

# Random Forest

In [ ]:

# Preparação dos dados
X_rf = df_microdados_fund.drop(columns=['CO_ENTIDADE', 'ABAND_FUND', 'CLASSE_ABAND'])
y_rf = df_microdados_fund['CLASSE_ABAND']

# Cria um objeto SMOTE
smote_rf = SMOTE()

# Gera amostras sintéticas para a classe minoritária
X_resampled_rf, y_resampled_rf = smote_rf.fit_resample(X_rf, y_rf)

# Encode labels
label_encoder_rf = LabelEncoder()
y_resampled_rf = pd.Series(label_encoder_rf.fit_transform(y_resampled_rf))

# Parâmetros para o RandomizedSearch
param_distributions_rf = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(10, 50),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'class_weight': ['balanced', 'balanced_subsample', None]
}

# RandomizedSearchCV para RandomForestClassifier
random_search_rf = RandomizedSearchCV(RandomForestClassifier(), param_distributions_rf, n_iter=20, cv=3, scoring='f1_weighted', n_jobs=-1, verbose=3)
random_search_rf.fit(X_resampled_rf, y_resampled_rf)

best_model_rf = random_search_rf.best_estimator_

# Avaliação adicional usando cross_validate com mais dobras (e.g., 10 dobras)
scoring_rf = {
    'accuracy': 'accuracy',
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score, average='weighted'),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
    'roc_auc_ovr': make_scorer(roc_auc_score, multi_class='ovr', average='weighted', needs_proba=True)
}

# Stratified K-Fold Cross Validation
skfold_rf = StratifiedKFold(n_splits=10, shuffle=True)

# Usar cross_validate para calcular as métricas de desempenho
results_rf = cross_validate(best_model_rf, X_resampled_rf, y_resampled_rf, cv=skfold_rf, scoring=scoring_rf)

# Cálculo das médias das métricas
mean_accuracy_rf = np.mean(results_rf['test_accuracy'])
mean_precision_rf = np.mean(results_rf['test_precision_weighted'])
mean_recall_rf = np.mean(results_rf['test_recall_weighted'])
mean_f1_rf = np.mean(results_rf['test_f1_weighted'])
mean_auc_rf = np.mean(results_rf['test_roc_auc_ovr'])

print("Média de Accuracy:", mean_accuracy_rf)
print("Média de Precision:", mean_precision_rf)
print("Média de Recall:", mean_recall_rf)
print("Média de F1-score:", mean_f1_rf)
print("Média de AUC-ROC:", mean_auc_rf)

print(best_model_rf)

In [ ]:

# Preparação dos dados
X_rf = df_microdados_med.drop(columns=['CO_ENTIDADE', 'ABAND_MED', 'CLASSE_ABAND'])
y_rf = df_microdados_med['CLASSE_ABAND']

# Cria um objeto SMOTE
smote_rf = SMOTE()

# Gera amostras sintéticas para a classe minoritária
X_resampled_rf, y_resampled_rf = smote_rf.fit_resample(X_rf, y_rf)

# Encode labels
label_encoder_rf = LabelEncoder()
y_resampled_rf = pd.Series(label_encoder_rf.fit_transform(y_resampled_rf))

# Parâmetros para o RandomizedSearch
param_distributions_rf = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(10, 50),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'class_weight': ['balanced', 'balanced_subsample', None]
}

# RandomizedSearchCV para RandomForestClassifier
random_search_rf = RandomizedSearchCV(RandomForestClassifier(), param_distributions_rf, n_iter=20, cv=3, scoring='f1_weighted', n_jobs=-1, verbose=3)
random_search_rf.fit(X_resampled_rf, y_resampled_rf)

best_model_rf = random_search_rf.best_estimator_

# Avaliação adicional usando cross_validate com mais dobras (e.g., 10 dobras)
scoring_rf = {
    'accuracy': 'accuracy',
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score, average='weighted'),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
    'roc_auc_ovr': make_scorer(roc_auc_score, multi_class='ovr', average='weighted', needs_proba=True)
}

# Stratified K-Fold Cross Validation
skfold_rf = StratifiedKFold(n_splits=10, shuffle=True)

# Usar cross_validate para calcular as métricas de desempenho
results_rf = cross_validate(best_model_rf, X_resampled_rf, y_resampled_rf, cv=skfold_rf, scoring=scoring_rf)

# Cálculo das médias das métricas
mean_accuracy_rf = np.mean(results_rf['test_accuracy'])
mean_precision_rf = np.mean(results_rf['test_precision_weighted'])
mean_recall_rf = np.mean(results_rf['test_recall_weighted'])
mean_f1_rf = np.mean(results_rf['test_f1_weighted'])
mean_auc_rf = np.mean(results_rf['test_roc_auc_ovr'])

print("Média de Accuracy:", mean_accuracy_rf)
print("Média de Precision:", mean_precision_rf)
print("Média de Recall:", mean_recall_rf)
print("Média de F1-score:", mean_f1_rf)
print("Média de AUC-ROC:", mean_auc_rf)

print(best_model_rf)

# K-means

### Método do cotovelo


In [ ]:

# Preparação dos dados
X_kmeans = df_microdados_fund.drop(columns=['CO_ENTIDADE', 'ABAND_FUND', 'CLASSE_ABAND'])

# Padronização dos dados
scaler = StandardScaler()
X_scaled_kmeans = scaler.fit_transform(X_kmeans)

# Determinando o número ideal de clusters com o Método do Cotovelo
inertia = []
K = range(2, 25)
for k in K:
    kmeans = KMeans(n_clusters=k, verbose=1)
    kmeans.fit(X_scaled_kmeans)
    inertia.append(kmeans.inertia_)

# Plotando o gráfico do Método do Cotovelo
plt.figure(figsize=(8, 6))
plt.plot(K, inertia, 'bx-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inertia')
plt.title('Método do Cotovelo para encontrar o número ideal de clusters')
plt.show()

# Avaliando diferentes números de clusters usando Silhouette Score
silhouette_scores = []
best_n = None
best_score = -2

for k in K[1:]:
    kmeans = KMeans(n_clusters=k, random_state=13)
    kmeans.fit(X_scaled_kmeans)
    labels = kmeans.labels_
    silhouette_score_value = silhouette_score(X_scaled_kmeans, labels)
    silhouette_scores.append(silhouette_score_value)

    if silhouette_score_value > best_score:
        best_score = silhouette_score_value
        best_n = k

print(f'Melhor n: {best_n} e melhor score: {best_score}')

# Plotando o gráfico do Silhouette Score
plt.figure(figsize=(8, 6))
plt.plot(K[1:], silhouette_scores, 'bx-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score para diferentes números de clusters')
plt.show()

# Treinando o modelo K-means com o número ideal de clusters
num_clusters = best_n  # Substituído pelo número ideal encontrado
kmeans = KMeans(n_clusters=num_clusters, random_state=13)
kmeans.fit(X_scaled_kmeans)

labels_kmeans = kmeans.predict(X_scaled_kmeans)
silhouette_avg_kmeans = silhouette_score(X_scaled_kmeans, labels_kmeans)
print("Média da Pontuação de Silhueta para K-means:", silhouette_avg_kmeans)

# Visualização dos clusters
# Utilizando PCA para reduzir a dimensionalidade para 2D
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled_kmeans)

plt.figure(figsize=(10, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=labels_kmeans, palette='viridis', s=50, alpha=0.6)
plt.title('Visualização dos Clusters com K-Means')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Clusters')
plt.show()


In [ ]:
# Preparação dos dados
X_kmeans = df_microdados_med.drop(columns=['CO_ENTIDADE', 'ABAND_MED', 'CLASSE_ABAND'])

# Padronização dos dados
scaler = StandardScaler()
X_scaled_kmeans = scaler.fit_transform(X_kmeans)

# Determinando o número ideal de clusters com o Método do Cotovelo
inertia = []
K = range(2, 25)
for k in K:
    kmeans = KMeans(n_clusters=k, verbose=1)
    kmeans.fit(X_scaled_kmeans)
    inertia.append(kmeans.inertia_)

# Plotando o gráfico do Método do Cotovelo
plt.figure(figsize=(8, 6))
plt.plot(K, inertia, 'bx-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inertia')
plt.title('Método do Cotovelo para encontrar o número ideal de clusters')
plt.show()

# Avaliando diferentes números de clusters usando Silhouette Score
silhouette_scores = []
best_n = None
best_score = -2

for k in K[1:]:
    kmeans = KMeans(n_clusters=k, random_state=13)
    kmeans.fit(X_scaled_kmeans)
    labels = kmeans.labels_
    silhouette_score_value = silhouette_score(X_scaled_kmeans, labels)
    silhouette_scores.append(silhouette_score_value)

    if silhouette_score_value > best_score:
        best_score = silhouette_score_value
        best_n = k

print(f'Melhor n: {best_n} e melhor score: {best_score}')

# Plotando o gráfico do Silhouette Score
plt.figure(figsize=(8, 6))
plt.plot(K[1:], silhouette_scores, 'bx-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score para diferentes números de clusters')
plt.show()

# Treinando o modelo K-means com o número ideal de clusters
num_clusters = best_n  # Substituído pelo número ideal encontrado
kmeans = KMeans(n_clusters=num_clusters, random_state=13)
kmeans.fit(X_scaled_kmeans)

labels_kmeans = kmeans.predict(X_scaled_kmeans)
silhouette_avg_kmeans = silhouette_score(X_scaled_kmeans, labels_kmeans)
print("Média da Pontuação de Silhueta para K-means:", silhouette_avg_kmeans)

# Visualização dos clusters
# Utilizando PCA para reduzir a dimensionalidade para 2D
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled_kmeans)

plt.figure(figsize=(10, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=labels_kmeans, palette='viridis', s=50, alpha=0.6)
plt.title('Visualização dos Clusters com K-Means')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Clusters')
plt.show()


### Random Search

In [ ]:
# Preparação dos dados
X_kmeans = df_microdados_fund.drop(columns=['CO_ENTIDADE', 'ABAND_FUND', 'CLASSE_ABAND'])
y_kmeans = df_microdados_fund['CLASSE_ABAND']  # Variável alvo para comparação

# Padronização dos dados
scaler = StandardScaler()
X_scaled_kmeans = scaler.fit_transform(X_kmeans)

# Função customizada para calcular Silhouette Score
def silhouette_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    return silhouette_score(X, labels)

# Parâmetros para RandomizedSearchCV
param_distributions = {'n_clusters': range(3, 20)}

# RandomizedSearchCV para KMeans
random_search_kmeans = RandomizedSearchCV(
    KMeans(n_init=10),
    param_distributions,
    n_iter=10,
    scoring=silhouette_scorer,
    verbose=1  # Para acompanhar a execução
)
random_search_kmeans.fit(X_scaled_kmeans)

# Melhor modelo
best_kmeans = random_search_kmeans.best_estimator_
print(f'Melhor número de clusters: {best_kmeans.n_clusters}')

# Score do melhor modelo
best_score = random_search_kmeans.best_score_
print(f'Silhouette Score do melhor modelo: {best_score}')

# Previsão dos clusters
labels_kmeans = best_kmeans.predict(X_scaled_kmeans)

# Adicionar os labels dos clusters ao DataFrame original
df_clusters = df_microdados_fund.copy()
df_clusters['Cluster'] = labels_kmeans

# Comparar os clusters com a variável alvo
cluster_summary = df_clusters.groupby(['Cluster', 'CLASSE_ABAND']).size().unstack(fill_value=0)

print("Resumo dos clusters comparado com a variável alvo:")
print(cluster_summary)


In [ ]:
random_search_kmeans.best_score_